In [ ]:
# 1. No, they would return the same gradients, makes learning impossible

In [ ]:
# 2. Yes

In [ ]:
# 3. No 0/dead neurons, average output at 0 negates the vanishing gradients problem, self normalizing effect for networks with all dense layers

In [ ]:
# 4.
# SELU: networks with all dense layers
# leaky ReLU(s): everything else
# ReLU: simplicity
# tanh: for output layers with result -1 to 1
# logistic: for output layers with probability-like output
# softmax: for output layers with mutually exclusive results

In [ ]:
# 5. High momentum can cause it to roll pass optimum

In [ ]:
# 6. l1 regularization, manual forcing weights to 0, and tensorflow optimization all can produce sparse model

In [ ]:
# 7. Yes, it equals to training many networks. No
# Yes, it equals to predicting with many networks

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
# 8. Deep neural net with CIFAR 10

In [2]:
# Import dataset
from tensorflow.keras.datasets import cifar10

(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()
X_train_full.shape

(50000, 32, 32, 3)

In [3]:
X_valid, X_train = X_train_full[:10000], X_train_full[10000:]
y_valid, y_train = y_train_full[:10000], y_train_full[10000:]

In [4]:
# Scaling for non SELU activations
X_valid = X_valid.astype('float32') / 255
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [5]:
# Model architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization

model = Sequential()
model.add(Flatten(input_shape=[32, 32, 3]))
for i in range(0, 20):
    if i > 15 and i <19:
        model.add(Dropout(rate=0.25))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='elu', kernel_initializer='lecun_normal'))
model.add(Dense(10, activation='softmax'))

In [7]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

In [8]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 3072)              12288     
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               1

In [10]:
# define callback
from tensorflow.keras.callbacks import EarlyStopping

early_cb = EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
# 1 cycle scheduling learning rate
K = keras.backend

class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                last_iter=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate/10
        self.last_iterations = last_iter or iterations//10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate/1000
        self.iteration = 0
    
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return((rate2-rate1) * (self.iteration-iter1) / (iter2-iter1) + rate1)
    
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, 
                                     self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2*self.half_iteration,
                                    self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2*self.half_iteration, self.iterations,
                                    self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)   

In [11]:
n_epochs = 200
batch_size = 32

In [ ]:
onecycle = OneCycleScheduler(len(X_train) // batch_size*n_epochs, max_rate=0.005)

In [12]:
history = model.fit(X_train, y_train,
                   epochs=n_epochs,
                   batch_size=batch_size,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_cb])

Train on 40000 samples, validate on 10000 samples
Epoch 1/200
40000/40000 [==============================] - 26s 639us/sample - loss: 1.9900 - accuracy: 0.2776 - val_loss: 1.8556 - val_accuracy: 0.3345
Epoch 2/200
40000/40000 [==============================] - 19s 484us/sample - loss: 1.7759 - accuracy: 0.3659 - val_loss: 1.7102 - val_accuracy: 0.3957
Epoch 3/200
40000/40000 [==============================] - 20s 496us/sample - loss: 1.6950 - accuracy: 0.3971 - val_loss: 1.6355 - val_accuracy: 0.4282
Epoch 4/200
40000/40000 [==============================] - 25s 631us/sample - loss: 1.6412 - accuracy: 0.4185 - val_loss: 1.5677 - val_accuracy: 0.4486
Epoch 5/200
40000/40000 [==============================] - 26s 662us/sample - loss: 1.5896 - accuracy: 0.4403 - val_loss: 1.5209 - val_accuracy: 0.4582
Epoch 6/200
40000/40000 [==============================] - 27s 666us/sample - loss: 1.5543 - accuracy: 0.4562 - val_loss: 1.4750 - val_accuracy: 0.4759
Epoch 7/200
40000/40000 [=============

In [13]:
# MC Dropout prediction
y_probas = np.stack([model(X_test, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_pred = np.argmax(y_proba, axis=1)
y_pred

array([3, 9, 0, ..., 5, 4, 7], dtype=int64)

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.534